MINI 2Q UCC POWELL

In [ ]:
#loading libraries
import os
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
from scipy.optimize import curve_fit
from scipy.stats import shapiro
!pip install fitter
from fitter import Fitter, get_common_distributions, get_distributions
from scipy.stats import norm
from scipy.optimize import minimize
%matplotlib inline
from scipy.stats import lognorm

Exact Result: -1.0521545049632666 HF energy: -0.9974650419151752 Exact correlation energy: -0.05468946304809141

In [ ]:
with open("Exact_Sol_MINI_2Q_UCC_POWELL.txt","r") as f:
    Ref_Energy = f.read()
    
print(Ref_Energy)

In [ ]:
with open("MINI_2Q_UCC_POWELL.txt","r") as f:
    vqe_MINI = f.read().splitlines()
#    vqe_6G.remove("Vqe Result")
    vqe_MINI = list(map(float,vqe_MINI))
print(len(vqe_MINI))

In [ ]:
with plt.style.context('seaborn'):
    plt.hist(vqe_MINI,density=True,bins='auto')
    plt.xlabel('Energy [H]')
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
#Histogram of the percentage deviation from reference data for the tests
# orginal 100*(np.array(ens)-REFERENCE_energy)/REFERENCE_energy
with plt.style.context('seaborn'):
    #plt.hist(100*(float(Ref_Energy)-np.array(vqe_MINI))/float(Ref_Energy),density=True,bins='auto')
    plt.hist(100*(np.array(vqe_MINI)-float(Ref_Energy))/float(Ref_Energy),density=True,bins='auto')
    plt.xlabel('Percentage error compared to FCI energy [%]')
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
err =100*(np.array(vqe_MINI)-float(Ref_Energy))/float(Ref_Energy)

[s_fit,loc_fit,scale_fit] = scipy.stats.lognorm.fit(err)
[loc2_fit,scale2_fit] = scipy.stats.cauchy.fit(err)
x = np.linspace(np.min(err),np.max(err))

with plt.style.context('seaborn'):
    plt.hist(err,bins = 'auto',density = True) #plotting rating hist
    plt.plot(x,scipy.stats.lognorm.pdf(x,s_fit,loc=loc_fit,scale=scale_fit),label='Lognorm')
    plt.plot(x,scipy.stats.cauchy.pdf(x,loc=loc2_fit,scale=scale2_fit),label='Cauchy')
    plt.xlabel('Percentage error compared to FCI energy [%]')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

In [ ]:
with plt.style.context('seaborn'):
    f = Fitter(err, 
           distributions= get_common_distributions(),bins='auto')
#Using fitter to gather loads of different distributions and calculate aic to rank them 
    f.fit() #showing fits
    f.summary() #displaying summary table
    plt.xlabel('Percentage error compared to FCI energy [%]')
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
stat, p = shapiro(err)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')